## Analysis for Colorado River Basin

This is calculating the zonal statistics, wettest and driest picks, and land cover/use analysis. These are the steps for the analysis only, the outputs are in the `/Data/analysis-data` folder. Use this ONLY if necessary to modify for your own use. 

### Part 1: Zonal Statisitics

In [24]:
#Running packages and load files necessary
import geopandas as gpd
import rasterstats 
import numpy as np

huc4_path = '../Data/processed-data/shapefiles/CRB_HUC4.shp'

#read the huc4 shapefile
CRB_huc4 = gpd.read_file(huc4_path)

#List of years to go through 
years = [str(y) for y in range(1984, 2022,1)]

#values to save for dataframe 
zonal_data = {
    'year': [],             #Year 
    'huc4': [],             #huc4 id's 
    'huc_name': [],         #huc4 name 
    'count_total': [],      #number of total water pixels (permanent + seasonal)
    'count_permanent':[],   #number of permanent water pixels 
    'count_seasonal':[],    #number of seasonal water pixels
    'count_nowater': [],    #number of no water pixels
    'count_no_obs': [],     #number of no observation pixels 
    'count_huc': [],        #number of total pixels in the huc, all classificiations 
}

Loop through all the raster files for the years 1984 to 2021 so that we get the number of pixels that are permanent, seasonal, no water, no observation.  \
Pixels are categorized as: 
- 1: No Water 
- 2: Seasonal Water 
- 3: Permanent Water 
- 4: No Observation 

 Total Water is the sum of permanent and seasonal water pixels. 

In [25]:
for yr in years: 
    #import raster file
    raster_path = '../Data/processed-data/yearly_water_history-CRB/' + str(yr) + '_CRB.tif'
    
    #calculate zonal stats 
    count_water = rasterstats.zonal_stats(huc4_path, raster_path, 
                                          stats = ['count'],
                                          categorical = True, 
                                          geojson_out = True)
    
    #get the hucs in order 
    huc4_order = []
    for i in range(0,len(count_water)):
        huc4_id = count_water[i]['properties']['huc4']
        huc4_order.append(int(huc4_id))

    sorted_huc4_i = np.argsort(huc4_order)
    for i in sorted_huc4_i:
        huc4_id = count_water[i]['properties']['huc4']
        huc4_name = count_water[i]['properties']['name']
        count_no_water = count_water[i]['properties'][1] #counting no water pixels
        count_seasonal = count_water[i]['properties'][2] #seasonal
        count_perm = count_water[i]['properties'][3] #permanent
        count_no_obs = count_water[i]['properties'][4] #no observation 
        count_total_water = count_perm + count_seasonal
        count_huc = count_water[i]['properties']['count']

        #append the values to the data 
        zonal_data['huc4'].append(huc4_id)
        zonal_data['huc_name'].append(huc4_name)
        zonal_data['year'].append(yr)
        zonal_data['count_seasonal'].append(count_seasonal)
        zonal_data['count_permanent'].append(count_perm)
        zonal_data['count_total'].append(count_total_water)
        zonal_data['count_no_obs'].append(count_no_obs)
        zonal_data['count_nowater'].append(count_no_water)
        zonal_data['count_huc'].append(count_huc)
    
    print("Finished calculating zonal stats for " + str(yr))


Finished calculating zonal stats for 1984
Finished calculating zonal stats for 1985
Finished calculating zonal stats for 1986
Finished calculating zonal stats for 1987
Finished calculating zonal stats for 1988
Finished calculating zonal stats for 1989
Finished calculating zonal stats for 1990
Finished calculating zonal stats for 1991
Finished calculating zonal stats for 1992
Finished calculating zonal stats for 1993
Finished calculating zonal stats for 1994
Finished calculating zonal stats for 1995
Finished calculating zonal stats for 1996
Finished calculating zonal stats for 1997
Finished calculating zonal stats for 1998
Finished calculating zonal stats for 1999
Finished calculating zonal stats for 2000
Finished calculating zonal stats for 2001
Finished calculating zonal stats for 2002
Finished calculating zonal stats for 2003
Finished calculating zonal stats for 2004
Finished calculating zonal stats for 2005
Finished calculating zonal stats for 2006
Finished calculating zonal stats f

In [28]:
import pandas as pd 
## Create Dataframe from the data 
zonal_df = pd.DataFrame(zonal_data)

########
# Calculating Area 
#  (number of pixels) * (pixel area = 30x30 m^2) / 1e6 to convert to km 
########

#size of pixel 
pixel_area = 30**2 #(meters^2) from literature
#convert to kilometers
km_scale = 1e6


#add calculations of area 
zonal_df['area_total_water']=(zonal_df.count_total * pixel_area) / km_scale
zonal_df['area_permanent']=(zonal_df.count_permanent * pixel_area) / km_scale
zonal_df['area_seasonal']=(zonal_df.count_seasonal * pixel_area) / km_scale
zonal_df['area_nowater'] = (zonal_df.count_nowater * pixel_area) /km_scale 

In [29]:
zonal_df

,year,huc4,huc_name,count_total,count_permanent,count_seasonal,count_nowater,count_no_obs,count_huc,area_total_water,area_permanent,area_seasonal,area_nowater
0,1984,1401,Colorado Headwaters,231912,178710,53202,32900,42481510,42746322,208.7208,160.8390,47.8818,29.6100
1,1984,1402,Gunnison,134992,103707,31285,16677,34231573,34383242,121.4928,93.3363,28.1565,15.0093
2,1984,1403,Upper Colorado-Dolores,74315,43541,30774,33235,35615233,35722783,66.8835,39.1869,27.6966,29.9115
3,1984,1404,Great Divide-Upper Green,624642,519318,105324,83780,92536757,93245179,562.1778,467.3862,94.7916,75.4020
4,1984,1405,White-Yampa,108011,50635,57376,43236,58191249,58342496,97.2099,45.5715,51.6384,38.9124
...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,2021,1503,Lower Colorado,443470,390309,53161,77626,69292021,69813117,399.1230,351.2781,47.8449,69.8634
566,2021,1504,Upper Gila,48195,5667,42528,109598,60540684,60698477,43.3755,5.1003,38.2752,98.6382
567,2021,1505,Middle Gila,70965,15575,55390,54920,66960197,67086082,63.8685,14.0175,49.8510,49.4280
568,2021,1506,Salt,228736,160596,68140,88507,54369575,54686818,205.8624,144.5364,61.3260,79.6563


Save dataframe as .csv file for future use.

In [30]:
zonal_df.to_csv('../Data/analysis-data/huc4_zonal_stats.csv', index=False)